In [ ]:
import torch
import numpy as np
import random

In [ ]:
!pip install datasets evaluate --upgrade
!python -m spacy download de_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is

In [ ]:
import spacy
import datasets
import tqdm
import torchtext
import evaluate
import torch.optim as optim

In [ ]:
seed = 42
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
from torchtext.datasets import Multi30k

/usr/local/lib/python3.10/dist-packages/torchtext/datasets/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [ ]:
!pip install torchdata

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!pip install 'portalocker>=2.0.0'

In [ ]:
import portalocker

In [ ]:
train_iter, valid_iter, test_iter = Multi30k(
    root='.data', split=('train', 'valid', 'test'), language_pair=('de', 'en')
)

/usr/local/lib/python3.10/dist-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


In [ ]:
src_sentence, tgt_sentence = next(iter(train_iter))
print(src_sentence,tgt_sentence)

Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche. Two young, White males are outside near many bushes.


In [ ]:
de_nlp = spacy.load('de_core_news_sm')
en_nlp = spacy.load('en_core_web_sm')

In [ ]:
def tokenize_data(en_nlp, de_nlp, max_length, lower, sos_token, eos_token, data) -> dict:
  en_text, de_text = data  # Unpack the tuple
  en_tokens = [token.text for token in en_nlp.tokenizer(en_text)][:max_length]
  de_tokens = [token.text for token in de_nlp.tokenizer(de_text)][:max_length]

  if lower:
    en_tokens = [i.lower() for i in en_tokens]
    de_tokens = [i.lower() for i in de_tokens]

  en_tokens = [sos_token] + en_tokens + [eos_token]
  de_tokens = [sos_token] + de_tokens + [eos_token]

  return {'en_tokens':en_tokens, 'de_tokens':de_tokens}

In [ ]:
from functools import partial

In [ ]:
max_length = 512
lower = True
eos_token = '<EOS>'
sos_token = '<SOS>'

fn_kwargs = {
    'en_nlp': en_nlp,
    'de_nlp': de_nlp,
    'max_length': max_length,
    'lower': lower,
    'sos_token': sos_token,
    'eos_token': eos_token
}

train_data = train_iter.map(partial(tokenize_data, en_nlp, de_nlp, max_length, lower, sos_token, eos_token))
test_data = test_iter.map(partial(tokenize_data, en_nlp, de_nlp, max_length, lower, sos_token, eos_token))
valid_data = valid_iter.map(partial(tokenize_data, en_nlp, de_nlp, max_length, lower, sos_token, eos_token))

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [ ]:
print(type(train_data))

<class 'torch.utils.data.datapipes.iter.callable.MapperIterDataPipe'>


In [ ]:
def yield_tokens(data_iter):
    for item in data_iter:
        yield item['de_tokens']  # Adjust this key based on the structure of your data

de_vocab = build_vocab_from_iterator(yield_tokens(train_data), min_freq=1)
de_vocab.insert_token('<unk>', 0)  # Add <unk> token at index 0
de_vocab.set_default_index(0)  # Set default index to 0

/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/iter/combining.py:337: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [ ]:
def yield_tokens_en(data_iter):
    for item in data_iter:
        yield item['en_tokens']  # Adjust this key based on the structure of your data

en_vocab = build_vocab_from_iterator(yield_tokens_en(train_data), min_freq=1)
en_vocab.insert_token('<unk>', 0)  # Add <unk> token at index 0
en_vocab.set_default_index(0)  # Set default index to 0

In [ ]:
en_vocab.get_stoi()

{'’em': 17978,
 'ürde': 17977,
 'üppiges': 17976,
 'übungsstunde': 17974,
 'übungsmatte': 17972,
 'übungsgrün': 17971,
 'übrigen': 17970,
 'überzeugen': 17968,
 'überwältigt': 17967,
 'überwiegende': 17963,
 'übertriebene': 17961,
 'überstehendes': 17960,
 'überspringt': 17959,
 'überschwemmung': 17958,
 'überschwemmte': 17957,
 'überschlags': 17956,
 'überschlagenen': 17955,
 'überschaubaren': 17954,
 'überschatteten': 17953,
 'überraschten': 17952,
 'überrascht': 17951,
 'übermalt': 17948,
 'übermalen': 17947,
 'überlegen': 17945,
 'überholt': 17943,
 'übergießt': 17939,
 'übergeschlagenen': 17938,
 'überfülltes': 17935,
 'überfüllter': 17934,
 'überfüllte': 17933,
 'übereinstimmenden': 17931,
 'überdimensioniertem': 17929,
 'überdachungen': 17927,
 'östlichen': 17922,
 'ölverseuchung': 17918,
 'ölpumpe': 17916,
 'ölkanne': 17915,
 'öffentlich': 17913,
 'äußerste': 17910,
 'äteres': 17907,
 'ärzten': 17904,
 'ärmell': 17902,
 'ändern': 17899,
 'ältliche': 17898,
 'älterem': 17896,
 '

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [ ]:
def tokens_to_indices(tokens, vocab):
    return [vocab.__getitem__(token) if isinstance(token, str) else 0 for token in tokens]

def collate_fn(batch):
    src_batch = [torch.tensor(tokens_to_indices(item['en_tokens'], en_vocab)) for item in batch]
    trg_batch = [torch.tensor(tokens_to_indices(item['de_tokens'], de_vocab)) for item in batch]

    # Pad sequences to the maximum length in the batch
    src_batch = pad_sequence(src_batch, padding_value=0, batch_first=True)
    trg_batch = pad_sequence(trg_batch, padding_value=0, batch_first=True)

    return {'en_tokens': src_batch.to(device), 'de_tokens': trg_batch.to(device)}

In [ ]:
BATCH_SIZE = 128

train_iterator = DataLoader(train_data, batch_size = BATCH_SIZE, collate_fn=collate_fn)
valid_iterator = DataLoader(valid_data, batch_size = BATCH_SIZE, collate_fn = collate_fn)
test_iterator = DataLoader(test_data, batch_size = BATCH_SIZE, collate_fn = collate_fn)

In [ ]:
class Encoder(torch.nn.Module):
  def __init__(self, input_size, hidden_size, embedding_size):
    super().__init__()
    self.hidden_size = hidden_size
    self.embedding = torch.nn.Embedding(input_size, embedding_size)
    self.dropout = torch.nn.Dropout(0.2)
    self.rnn = torch.nn.GRU(embedding_size, self.hidden_size)

  def forward(self, input):
    #self.input = input.unsqueeze(0)
    #print("C", input.shape)
    input = input.transpose(0, 1)
    self.embed = self.dropout(self.embedding(input))
    #print("B", self.embed.shape)
    output, hidden = self.rnn(self.embed)
    #print("A", output.shape, hidden.shape)
    return hidden

In [ ]:
class Decoder(torch.nn.Module):
  def __init__(self, output_size, embedding_size, hidden_size):
    super().__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.embedding = torch.nn.Embedding(output_size, embedding_size)
    self.dropout = torch.nn.Dropout(0.2)
    self.rnn = torch.nn.GRU(embedding_size+hidden_size, hidden_size)
    self.fc = torch.nn.Linear(embedding_size+hidden_size*2, output_size)

  def forward(self, input, context, hidden):
    #print(input.shape, hidden.shape)
    self.input = input.unsqueeze(0)
    embed = self.dropout(self.embedding(self.input))
    #print("A", input.shape, embed.shape, context.shape)
    self.embed = torch.concat([embed, context], dim = 2)
    output, hidden = self.rnn(self.embed, hidden)
    output = torch.concat((embed.squeeze(0), hidden.squeeze(0), context.squeeze(0)), dim = 1)
    #print("B", self.embed.shape, hidden.shape, context.shape)
    #print(output.shape)
    prediction = self.fc(output)
    return prediction, hidden

In [ ]:
class Seq2Seq(torch.nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()
    self.device = device
    self.encoder = encoder
    self.decoder = decoder

  def forward(self, src, trg, teacher_forcing_ratio = 0.5):
    self.batch_size = trg.shape[0]
    #print("Trg", trg.shape)
    trg_len = trg.shape[1]
    self.trg_vocab_size = self.decoder.output_size

    outputs = torch.zeros(trg_len, self.batch_size, self.trg_vocab_size).to(self.device)

    hidden = self.encoder(src)
    context = hidden

    input = trg[:, 0]

    for t in range(1, trg_len):
      output, hidden = self.decoder(input, context, hidden)
      outputs[t] = output

      teacher_force = random.random() < teacher_forcing_ratio

      top1 = output.argmax(1)
      #print("D", top1.shape, trg[:, t].shape)
      input = trg[:, t] if teacher_force else top1

    return outputs

In [ ]:
input_size = len(en_vocab)
output_size = len(de_vocab)
hidden_size = 512
embedding_size = 256

encoder = Encoder(input_size, hidden_size, embedding_size)
decoder = Decoder(output_size, embedding_size, hidden_size)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
def init_weights(m):
  for name, param in model.named_parameters():
    torch.nn.init.normal_(param.data, mean = 0, std=0.01)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(17980, 256)
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn): GRU(256, 512)
  )
  (decoder): Decoder(
    (embedding): Embedding(10358, 256)
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn): GRU(768, 512)
    (fc): Linear(in_features=1280, out_features=10358, bias=True)
  )
)

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
trg_idx = de_vocab.lookup_indices(['<pad>'])
lossfn = torch.nn.CrossEntropyLoss(ignore_index = trg_idx[0])

In [ ]:
def train(model, iterator, optimizer, lossfn, clip):
  model.train()

  epoch_loss = 0
  count = 0
  for i, batch in enumerate(iterator):
    #print(len(batch))
    src = batch['en_tokens']
    trg = batch['de_tokens']
    #print(src.shape, trg.shape)
    optimizer.zero_grad()

    pred = model(src, trg)
    output_dim = pred.shape[-1]
    pred = pred[1:].view(-1, output_dim)
    trg = trg.transpose(0, 1)
    trg = trg[1:]
    trg = trg.reshape(-1)
    #print(pred.shape, trg.shape)
    loss = lossfn(pred, trg)

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()

    epoch_loss += loss.item()
    count += 1

  return epoch_loss/(count*128)

In [ ]:
import time

In [ ]:
def evaluate(model, iterator, lossfn):
  model.eval()
  epoch_loss = 0
  count = 0
  with torch.no_grad():
    for i, batch in enumerate(iterator):
      src = batch['en_tokens']
      trg = batch['de_tokens']

      pred = model(src, trg)
      output_dim = pred.shape[-1]
      pred = pred[1:].view(-1, output_dim)
      trg = trg.transpose(0, 1)
      trg = trg[1:]
      trg = trg.reshape(-1)

      loss = lossfn(pred, trg)

      epoch_loss += loss.item()
      count += 1

  return epoch_loss/(count*128)

In [ ]:
import math

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, lossfn, CLIP)
    valid_loss = evaluate(model, valid_iterator, lossfn)

    end_time = time.time()

    #epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')

    #print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

	Train Loss: 0.029 | Train PPL:   1.029
	 Val. Loss: 0.028 |  Val. PPL:   1.029
	Train Loss: 0.026 | Train PPL:   1.026
	 Val. Loss: 0.027 |  Val. PPL:   1.027
	Train Loss: 0.023 | Train PPL:   1.023
	 Val. Loss: 0.026 |  Val. PPL:   1.026
	Train Loss: 0.020 | Train PPL:   1.021
	 Val. Loss: 0.025 |  Val. PPL:   1.026
	Train Loss: 0.018 | Train PPL:   1.019
	 Val. Loss: 0.024 |  Val. PPL:   1.024
	Train Loss: 0.017 | Train PPL:   1.017
	 Val. Loss: 0.024 |  Val. PPL:   1.024
	Train Loss: 0.015 | Train PPL:   1.015
	 Val. Loss: 0.023 |  Val. PPL:   1.024
	Train Loss: 0.014 | Train PPL:   1.014
	 Val. Loss: 0.024 |  Val. PPL:   1.024
	Train Loss: 0.013 | Train PPL:   1.013
	 Val. Loss: 0.025 |  Val. PPL:   1.025
	Train Loss: 0.012 | Train PPL:   1.012
	 Val. Loss: 0.023 |  Val. PPL:   1.024


In [56]:
print("Done!!")

Done!!
